In [1]:
# Deletes previous images
import os

for i in range(count):
    os.remove("Resources/Scanned/Plate_"+str(i)+".jpg")


NameError: name 'count' is not defined

In [2]:
# _________________________________________________Number Plate Recognition_______________________________________________________

import cv2

frameWidth = 640
frameHeight = 480
# Import the model
PlateCascade = cv2.CascadeClassifier("Resources/haarcascade.xml")
minArea = 500
color = (0,0,255)
count = 0

vid = cv2.VideoCapture(1)
vid.set(3, frameWidth)
vid.set(4, frameHeight)
vid.set(10, 100)

while True:
    success, img = vid.read()
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    numberPlates = PlateCascade.detectMultiScale(imgGray, 1.1, 4)

    for (x, y, w, h) in numberPlates:
        area=w*h
        if area > minArea:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 4)
            cv2.putText(img,"Number Plate",(x,y-5),cv2.FONT_HERSHEY_COMPLEX,1,color,2)
            imgRoi = img[y:y+h, x:x+w]
            cv2.imshow("ROI", imgRoi)


    cv2.imshow("Output", img)
    
    k = cv2.waitKey(1)
    if k & 0xFF==ord('s'):
        cv2.imwrite("Resources/Scanned/Plate_"+str(count)+".jpg",imgRoi)
        cv2.rectangle(img,(0,200),(640,300),(0,255,0),cv2.FILLED)
        cv2.putText(img,"No.Plate Saved",(100,265),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
        cv2.imshow("Output", img)
        count += 1
        cv2.waitKey(500)
    elif k == 27:
        break
    elif k == -1:
        continue
        
cv2.destroyAllWindows()  
cv2.waitKey(27)

#_____________________________________________Text Recognition from Number PLate_______________________________________________________

# !pip install pytesseract
from pytesseract import pytesseract
import string
try:
    from PIL import Image
except ImportError:
    import Image    
import numpy as np
import cv2
pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

#-------------------------------Function---------------------------------

def preprocessing_num_plate(extractedInformation):

    
    """Function removes special character and gives number plate."""
    
    strt=[]
    end=0
    spl_char = ['|', '“', '}', '{', ' ', ':','(',')','/',']','[','~','*','.','"','\'','a','b','c','d','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    n = ['1','2','3','4','5','6','7','8','9','0']
    d = list(string.ascii_uppercase)
    result = ''
    for i in range(len(list(extractedInformation))):
        if extractedInformation[i] not in spl_char:
            
            if extractedInformation[i] in  d:
                result = result + extractedInformation[i].lower()
                strt.append(i)
            if extractedInformation[i] in n:
                result = result + extractedInformation[i]
                end = i
    if strt != None:
        if len(strt) != 0 and end != 0:

            return result
        else:
            return
    return 0

def common_num_plate(lists_np):
    
    """Function returns most common value in photos"""
    
    dict_np = {}
    maxi = 0
    
    for i in lists_np:
        if i not in dict_np.keys():
            dict_np[i] = 0
        elif i in dict_np.keys():
            dict_np[i] = dict_np[i] + 1
    
    for key, value in dict_np.items():
        if maxi < value:
            k_maxi = key
    try:       
        return k_maxi
    except:
        return 'Try again'
#------------------------------------------------------------------------
pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# count = 10
start = 0
img_num = count -1
num_plates = []
while start <= img_num:
    img_path = r"Resources/Scanned/Plate_" + str(start)+".jpg"
    
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    extractedInformation = pytesseract.image_to_string(gray)
#     print(extractedInformation)
    if preprocessing_num_plate(extractedInformation) != None:
        if (len(preprocessing_num_plate(extractedInformation)) != 0) and (len(preprocessing_num_plate(extractedInformation)) in [9,10]):
                                    num_plates.append(preprocessing_num_plate(extractedInformation))
                                
    start=start +1
# print(num_plates)
num_plate = common_num_plate(num_plates)
# print(num_plate)
    

    
    
    
#______________________________________________________Web Scraping______________________________________________________________

from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin
session = HTMLSession()


##-------------------------------Function---------------------------------

def get_all_forms(url):
    
    """Returns all form tags found on a web page's `url` """
    
    res = session.get(url)
    soup = BeautifulSoup(res.html.html, "html.parser")
    return soup.find("form")

def get_form_details(form):
    
    """Returns the HTML details of a form,
    including action, method and list of form controls (inputs, etc)"""
    
    details = {}
    action = form.attrs.get("action")
    method = form.attrs.get("method", "get").lower()
    inputs = []
    
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        input_value =input_tag.attrs.get("value", "")
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    
    return details


def url_maker(data, url):
    state_in_short = ''
    numbers_in_data = ''
    count = 0
    
    for i in data:
        if count < 2:
            state_in_short = state_in_short + i
        if count >= 2 and count < 4 and i.isnumeric():
            numbers_in_data = numbers_in_data + i
        count += 1
    
    
    if int(numbers_in_data) < 10:
        numbers_in_data = str(numbers_in_data)
        if len(numbers_in_data) != 2:
            numbers_in_data = '0' + numbers_in_data
        else:
            numbers_in_data  = str(numbers_in_data)
    state_dict = {'ka':'karnataka','hr':'haryana','mh':'maharashtra','tn':'tamil-nadu','dl':'delhi'}

    return url[:-1] + '-' + state_dict.get(state_in_short) + '-' + state_in_short + '-' + numbers_in_data +'/'
    
#------------------------------------------------------------------------

url = "https://www.cars24.com/rto-vehicle-registration-details/"

forms = get_all_forms(url)
form_details = get_form_details(forms)

data = {}

for input_tag in form_details["inputs"]:
    if input_tag["type"] == "hidden":
        data[input_tag["name"]] = input_tag["value"]
    elif input_tag["type"] != "submit":
        value = num_plate
        data[input_tag["name"]] = value
        
post_url = url_maker(data[input_tag["name"]],url)        
print(post_url)
url = urljoin(url, form_details["action"])
# print(url)
if form_details["method"] == "post":
    res = session.get(url)
elif form_details["method"] == "get":
    url = post_url
    res = session.get(url)
# print(res)

# # https://www.cars24.com/rto-vehicle-registration-details/
# # https://www.cars24.com/rto-vehicle-registration-details-karnataka-ka-20/


try:
    soup = BeautifulSoup(res.content, "html.parser")
    ul = soup.find_all('span')

    li=[]
    for i in ul:
          if i.find('a') not in i and i.find('img') not in i and i.find('div') not in i:
                li.append(i)
        
    li = li[1:13]
#     print(li)
except:
    print('Page Not Found')





import IPython

strTable = """<html>
            <head><link rel="stylesheet" href="Output/table.css"></head>
           <body><table><tr><th>Details</th><th>Value</th></tr>"""
detail_list = ['Number', 'City', 'State', 'Phone', 'Email ID', 'Address']
for i in li:
    if i.text in detail_list:            
             strRW = "<tr><td>"+i.text+ "</td>"           
    else:
        strRW = "<td>"+i.text+"</td></tr>"

    strTable = strTable+strRW

strTable = strTable+"</table></body></html>"

hs = open("Output/table.html", 'w')
hs.write(strTable)
hs.close()

IPython.display.HTML(filename='Output/table.html')


ValueError: invalid literal for int() with base 10: ''